# Shank of each neuron

We want to know on which shank a neuron was recorded from.

We already have the channel on which the neuron had its largest spike waveform amplitude.

We also have the shank number for each channel.

* channel_map.npy
* channel_positions.npy
* shank_layer.csv

In [1]:
%reload_ext autoreload
%autoreload 2
%run setup_project.py

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects


In [2]:
load_spike_train_project(sSesList)

100%|███████████████████████████████████████████| 39/39 [00:08<00:00,  4.85it/s]


In [3]:
def shank_neuron(ses):
    """
    Save a csv file with the cell_id, shank and layer for each neuron.
    
    """
    cm = np.load(ses.path+"/channel_map.npy")
    cp = np.load(ses.path+"/channel_positions.npy")
    sl = pd.read_csv(ses.path+"/shank_layer.csv")
    df_map = pd.DataFrame({"id":cm[:,0],
                         "shank":cp[:,0].astype(int),
                         "chanOnShank":cp[:,1]})
    clafn=(f"{ses.fileBase}.channel_largest_waveform_amplitude_2000.npy")
    neuronChan =np.load(clafn)
    
    if len(ses.cg.neuron_list) != neuronChan.shape[0]:
        print("problem with length of neuron list")
    if len([ ch  for ch in neuronChan if not np.any(df_map.id==ch)]) > 0:
        print("some channels with max amplitude waveforms are not in the channel_map")
        return
    shanks = np.array([ df_map.shank[df_map.id==ch].head(n=1).item()  for ch in neuronChan])
    layer = [ sl.layer[sl.shank==sh].item()  for sh in shanks]
    df = pd.DataFrame({"id":[ ses.name+"_"+n.name for n in ses.cg.neuron_list],
                 "shank":shanks,
                 "layer":layer})
   
    fn = (f"{ses.fileBase}.shank_neuron.csv")
    print("Saving ",fn) 
    df.to_csv(fn,index=False)

In [4]:
for i,ses in enumerate(sSesList):
    shank_neuron(ses)

Saving  /adata/projects/autopi_ca1/mn5824/mn5824-20112020-0107/mn5824-20112020-0107.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/mn5824-22112020-0107.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn5824/mn5824-24112020-0107/mn5824-24112020-0107.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn5824/mn5824-02122020-0106/mn5824-02122020-0106.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-28012021-0106/mn711-28012021-0106.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-30012021-0106/mn711-30012021-0106.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-31012021-0107/mn711-31012021-0107.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-01022021-0107/mn711-01022021-0107.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-02022021-0108/mn711-02022021-0108.shank_neuron.csv
Saving  /adata/projects/autopi_ca1/mn711/mn711-03022021-0107/mn711-03022021-0107.shank_neuron.csv
Saving  

In [5]:
listDf = [ pd.read_csv(ses.fileBase+".shank_neuron.csv") for ses in sSesList]
df = pd.concat(listDf)
fn=myProject.dataPath+"/results/shank_neuron.csv"
print("Saving in",fn)
df.to_csv(fn,index=False)

Saving in /adata/projects/autopi_ca1/results/shank_neuron.csv
